In [ ]:
# this section is just gettign all of the liabarys that are needed 
library(tidyverse)
library(polite) # this library is jsut a better version of the rvest library 
library(xml2) # makes it easier to work with HTML and XML from R
library(purrr) # to work with lists and map functions
library(glue) # to paste strings
library(rvest) # rvest makes scraping easier

In [12]:
# The Neat Places website URL
url <- "https://neatplaces.co.nz/places/christchurch-canterbury/eat-drink/cafes"

# The Neat Places website HTML
page <- read_html(url)

# Vector of Christchurch cafe names
# Scans the page HTML for the specified node and returns the text
cafe_name <- page %>% 
   html_nodes(".list-item-title") %>% 
   html_text()

# Dataframe of Christchurch cafe names
cafe_names <- tibble(cafe_name)

# Vector of Christchurch cafe descriptions
summary <- page %>%
  html_nodes(".summary") %>%
  html_text()

# Vector of links to each of the cafes
# Searches for specified node and HREFhref attribute to get links
link <- page %>% 
  html_nodes(".list-item a") %>% 
  html_attr('href')

# Dataframe of cafe names and related page link
link_table <- tibble(cafe_name, link)

In [31]:
# Function which outputs data for different places in nz as a search option
# Takes in a city in NZ and returns its avalable cafes
change_place <- function(place_name) {
    # Converts given place name to URL format
    url_name <- place_name %>%
        tolower() %>% 
        str_replace_all(" & ", "-") %>%
        str_replace_all(" ", "-") %>%
        str_replace_all("ō", "o") %>%
        str_replace_all("ā", "a")
    
    # The generated URL
    new_url <- glue("https://neatplaces.co.nz/places/{url_name}/eat-drink/cafes") 

    # The page HTML of the generated URL
    page <- read_html(new_url) 
    
    # Vector of the cafe names
    cafe_data <- page %>% 
        html_nodes(".list-item-title p") %>%
        html_text()

    return(cafe_data)
}     

In [14]:
# The AudioCulture website in html
audio_culture_page = read_html("https://www.audioculture.co.nz/music_index?category=Person")

# Profile names
Profiles = audio_culture_page %>% html_nodes(".skippy-col-link") %>% html_text()

# Profile links
Links = audio_culture_page %>% html_nodes(".skippy-col-link") %>% html_attr('href')

# Dataframe of profile and associated link
df = data.frame(Profiles, Links)

In [28]:
# Prints NZ music profiles based on chars of given string
create_profile_list = function(cafe_name) {
    # Removes special symbols and whitespace from string
    cleaned_string = gsub("([^A-Za-z0-9])+", "", 
        cafe_name %>% 
        str_replace_all("é", "e") %>%
        str_replace_all("ō", "o") %>%
        str_replace_all("ā", "a"))

    # Splits string into list for iteration
    char_list = strsplit(cleaned_string, "")[[1]]

    # The list of NZ profiles based on cafe name
    profile_list = data.frame()
    
    # Iterate through every char in the given string
    for (char in char_list) {
        # Converts char into upper case for REGEX pattern
        upper_char = toupper(char)

        # Converts char into lower case for REGEX pattern
        lower_char = tolower(char)

        # Creates list of profiles that match REGEX pattern (Starts with given char)
        match_list = grep(glue('^[{upper_char}{lower_char}]'), Profiles, value = TRUE)
        
        # If the match list is not empty choose random profile from match list
        if (length(match_list) != 0) {
                
            # Does at most 5 attempts to find at least 1 album from a profile
            for (i in 1:5) {
                # Randomly selected profile
                profile_match = sample(match_list, 1)

                # The profile's discography
                profile_discography = get_discography(profile_match)
                
                if (nrow(profile_discography) >= 1) {
                    break()
                }
            }

            # Sometimes profiles have empty discographies
            if (nrow(profile_discography) == 0) {
                profile_album = data.frame(Character = upper_char, 
                                           Profile = profile_match, 
                                           Album = NA_character_, 
                                           Featuring = NA_character_, 
                                           Year = NA_real_)
            } else {
                # profile_album_index = match(profile_discography[3] >= 2010)[2]


                # Randomly selects album index from discography
                profile_album_index = sample(nrow(profile_discography), 1) 

                # Create data frame entry for profile_lsit
                profile_album = merge((data.frame(Character = upper_char, Profile = profile_match)), 
                                      (profile_discography[profile_album_index,]))
            }
            
        } else {
            # If no match, create empty data frame entry for profile_list
            profile_album = data.frame(Character = upper_char, 
                                       Profile = NA_character_, 
                                       Album = NA_character_, 
                                       Featuring = NA_character_, 
                                       Year = NA_real_)
        }

        # Appends profile_list with new profile
        profile_list = rbind(profile_list, profile_album)
    }

    return(profile_list)
}

In [29]:
# Takes a profile name as input
# Returns given name's discography (currenly no error handling if name not in dataframe)
get_discography = function(name) {
    name_link = (df %>% filter(str_detect(Profiles, name)))[[2]][1]
    
    profile_discography_page = glue('https://www.audioculture.co.nz{name_link}/discography') %>% read_html()

    Album     = profile_discography_page %>% html_nodes("[class='header']") %>% html_text()
    Featuring = profile_discography_page %>% html_nodes("[class='body']")   %>% html_text()
    Year      = profile_discography_page %>% html_nodes("[class='year']")   %>% html_text() %>% as.numeric()
    
    # Replaces empty entries with NA
    Album[Album == ""] = NA_character_

    # Replaces empty entries with NA
    Featuring[Featuring == ""] = NA_character_

    # If the year length is less than 1000, replace with NA
    Year[Year < 1000] = NA_real_
    
    # Data frame of the given profile's discography
    discography_df = data.frame(Album, Featuring, Year)

    return(discography_df %>% filter(Year >= 1000))
}

In [34]:
selected_city   = "Auckland"
selected_cafes  = change_place(selected_city)
selected_cafe   = selected_cafes[2]
print(glue("City: {selected_city}"))
print(glue("Cafe: {selected_cafe}"))
create_profile_list(selected_cafe)

City: Auckland
Cafe: Völla Café


Character,Profile,Album,Featuring,Year
<chr>,<chr>,<chr>,<chr>,<dbl>
V,Val Murphy,My Way Of Singing,Val Murphy,1965
O,Opshop,{Fig.01. You Are Here,Opshop,2004
L,Living Force,Living Force,Living Force,1977
L,Loves Ugly Children,Personal World,Loves Ugly Children,1995
A,Andre Upston,Pretty Cool,Trip (10),2002
C,Cut Off Your Hands,Still Fond,Cut Off Your Hands,2007
A,Alastair Galbraith,Timebomb,Alastair Galbraith & Graeme Jefferies,1989
F,Flesh D-Vice,Hamburger II: The Night She Returned,Flesh D-Vice,1988
E,Edmund Cake,Downtown Puff,Edmund Cake,2005
